In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [3]:
train_generator = train_datagen.flow_from_directory(
        '/kaggle/input/chest-xray-pneumonia/chest_xray/train/',
        target_size=(128, 128),
        batch_size=8,
        class_mode='binary')

validation_generator = train_datagen.flow_from_directory(
        '/kaggle/input/chest-xray-pneumonia/chest_xray/test/',
        target_size=(128, 128),
        batch_size=8,
        class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [4]:
resnet101 = tf.keras.applications.resnet.ResNet50(include_top=False, weights=None,input_shape=(128, 128, 3), pooling='avg')
x1 = tf.keras.layers.Dense(1, activation='sigmoid')(resnet101.output)
resnet101_model = tf.keras.Model(resnet101.input, x1)

In [5]:
METRICS = [
        'accuracy',
    ]
    
resnet101_model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=METRICS
    )

In [6]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 20)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [7]:
history = resnet101_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[lr_scheduler]
)

Epoch 1/10
652/652 [==============================] - 155s 181ms/step - loss: 0.7022 - accuracy: 0.7406 - val_loss: 4.4002 - val_accuracy: 0.5000 - lr: 0.0100
Epoch 2/10
652/652 [==============================] - 79s 121ms/step - loss: 0.3067 - accuracy: 0.8662 - val_loss: 0.3916 - val_accuracy: 0.8478 - lr: 0.0089
Epoch 3/10
652/652 [==============================] - 79s 121ms/step - loss: 0.2674 - accuracy: 0.8855 - val_loss: 1.9427 - val_accuracy: 0.4054 - lr: 0.0079
Epoch 4/10
652/652 [==============================] - 78s 120ms/step - loss: 0.2380 - accuracy: 0.9016 - val_loss: 2.6401 - val_accuracy: 0.6410 - lr: 0.0071
Epoch 5/10
652/652 [==============================] - 79s 121ms/step - loss: 0.2199 - accuracy: 0.9135 - val_loss: 0.5758 - val_accuracy: 0.7740 - lr: 0.0063
Epoch 6/10
652/652 [==============================] - 78s 120ms/step - loss: 0.2086 - accuracy: 0.9147 - val_loss: 1.4616 - val_accuracy: 0.7099 - lr: 0.0056
Epoch 7/10
652/652 [==============================]

In [8]:
resnet101_model.save("ResNet101_xray_Pneumonia_50.h5")